### Chatbot

In [1]:
import random 

import joblib

import sys
import os


import random
random.seed(42)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve

import matplotlib.pyplot as plt

import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, concatenate, Dropout
from tensorflow.keras.models import Model

import scipy

In [5]:
df = pd.read_excel(r"database_intents.xlsx")

In [22]:
df.columns

Index(['Intent type', 'Sentence'], dtype='object')

In [26]:
class_names = df['Intent type'].unique()

In [6]:
# Usar get_dummies en vez de las 3 funciones
df_oh = pd.concat([df, pd.get_dummies(df['Intent type'])], axis = 1)

In [7]:
df_train, df_test = train_test_split(df_oh, train_size = 0.7, test_size = 0.3, random_state = 42,
                                    shuffle = True, stratify = df_oh['Intent type'])

In [9]:
from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(df_train['Sentence']).batch(128)
vectorizer.adapt(text_ds)

In [10]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'me', 'to', 'you']

In [12]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [15]:
embeddings_index = {}
with open('glove.6B.50d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [17]:
num_tokens = len(voc) + 2
embedding_dim = 50
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 146 words (13 misses)


In [18]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [28]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 50)          8050      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         32128     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)         82048 

In [36]:
X_train = vectorizer(np.array([[s] for s in df_train['Sentence']])).numpy()
X_test = vectorizer(np.array([[s] for s in df_test['Sentence']])).numpy()

y_train = np.array(df_train.iloc[:, 2:])
y_test = np.array(df_test.iloc[:, 2:])

In [40]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(85, 200)
(85, 6)
(37, 200)
(37, 6)


In [43]:
model.compile(
    loss="categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(X_train, y_train, batch_size=128, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
1/1 [==============================] - 1s 719ms/step - loss: 1.8267 - acc: 0.0706 - val_loss: 1.6713 - val_acc: 0.3243
Epoch 2/20
1/1 [==============================] - 0s 52ms/step - loss: 1.6772 - acc: 0.3176 - val_loss: 1.5199 - val_acc: 0.3243
Epoch 3/20
1/1 [==============================] - 0s 48ms/step - loss: 1.5215 - acc: 0.3647 - val_loss: 1.4517 - val_acc: 0.3243
Epoch 4/20
1/1 [==============================] - 0s 50ms/step - loss: 1.4588 - acc: 0.3765 - val_loss: 1.4327 - val_acc: 0.5405
Epoch 5/20
1/1 [==============================] - 0s 51ms/step - loss: 1.3963 - acc: 0.4941 - val_loss: 1.3925 - val_acc: 0.3784
Epoch 6/20
1/1 [==============================] - 0s 51ms/step - loss: 1.3285 - acc: 0.3882 - val_loss: 1.4931 - val_acc: 0.4054
Epoch 7/20
1/1 [==============================] - 0s 53ms/step - loss: 1.4103 - acc: 0.4471 - val_loss: 1.3736 - val_acc: 0.5405
Epoch 8/20
1/1 [==============================] - 0s 51ms/step - loss: 1.2942 - acc: 0.5176 - va

### Pruebas

In [44]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["Hi, how are you doing today?"]]
)

class_names[np.argmax(probabilities[0])]

'Search'

### No lo he probado en el chatbot, para hacer pruebas usar el módulo de arriba

# CHATBOT TEST

In [83]:
def chatbot():
    
    input_text = input()
    
    test = pd.DataFrame(data = {'Sentence': [input_text]})
    df_test_proc, test_proc = processing(test, cv = cv)
    scipy.sparse.csr_matrix.sort_indices(test_proc)
    
    gret_prob = mlp_greeting.predict(test_proc)
    search_prob = mlp_search.predict(test_proc)
    sugg_prob = mlp_suggestion.predict(test_proc)
    farewell_prob = mlp_farewell.predict(test_proc)
    
    probs = [gret_prob, search_prob, sugg_prob, farewell_prob]
    print(probs)
    idx = np.argmax(probs)
    
    if idx == 0:
        print("Esto es un saludo")
    elif idx == 1:
        print("Esto es una búsqueda")
    elif idx == 2:
        print("Esto es una sugerencia")
    else:
        print("Esto es una despedida")
        
#     print('¿Hemos acertado?')
    
#     respuesta = input()
#     if (respuesta == 'No' or respuesta == 'no'):
#         probs = np.delete(probs, idx)
#         idx_2 = np.argmax(probs)
        
#         if idx == 0:
#             if idx_2 == 0:
#                 print("Esto es una búsqueda")
#             else:
#                 print("Esto es una sugerencia")
#         elif idx == 1:
#             if idx_2 == 0:
#                 print("Esto es un saludo")
#             else:
#                 print("Esto es una sugerencia")
#         else:
#             if idx_2 == 0:
#                 print("Esto es un saludo")
#             else:
#                 print("Esto es una búsqueda")
#     else:
#         print('¡Genial! ¡Hemos acertado!')
    

In [96]:
chatbot()

hi
[array([[0.8040894]], dtype=float32), array([[0.00038779]], dtype=float32), array([[3.612887e-05]], dtype=float32), array([[0.00043699]], dtype=float32)]
Esto es un saludo


In [81]:
chatbot()

hi
[array([[0.7962549]], dtype=float32), array([[0.01487446]], dtype=float32), array([[0.0009267]], dtype=float32), array([[4.4460567e-06]], dtype=float32)]
Esto es un saludo


In [ ]:
chatbot()

In [ ]:
chatbot()

* Correctamente
 * La mayoría de saludos
 * I want to know about Data Science (búsqueda)
 * I want to know things connected to la Sagrada Familia (sugerencia)
 * Watcha doing'? (saludo)
 * Tell me things related to Data Science (sugerencia)

* Incorrectamente
 * What is Data Science (sugerencia)
 * Tell me things related to Napoleon (saludo)
 * Who is Cristobal Colon (saludo)



In [ ]:
chatbot()

* Opciones
 * Mostrar un comando *!options* que te de las opciones para hacer.
 * Detectar un intent
 * Mostrarlo al inicio


* Headers
 * Dar las opciones una vez se detecta intent de búsqueda búsqueda
   * *What is data science?* Y mostrarle los heades para qué elija qué queire saber 
 * Intent nuevo de header

In [ ]:
chatbot()

In [ ]:
def Intent_detection_function(keyboard):
    gret_prob = mlp_greeting.predict_proba(test_proc)
    search_prob = mlp_search.predict_proba(test_proc)
    sugg_prob = mlp_suggestion.predict_proba(test_proc)
    farewell_prob = mlp_farewell.predict_proba(test_proc)
    cv = joblib.load("vectorizer.pkl")


    input_text = pd.DataFrame(data={'Sentence': [keyboard]})
    _, test_proc = cv.transform(input_text['Sentence'])
    #df_test_proc, test_proc = processing(test, cv=cv)

    test_proc = test_proc.toarray()

    gret_prob = mlp_greeting.predict(test_proc)

    search_prob = mlp_search.predict(test_proc)
    sugg_prob = mlp_suggestion.predict(test_proc)
    fare_prob = mlp_farewell.predict(test_proc)


    probs = [gret_prob, search_prob, sugg_prob, fare_prob]
    print(probs)
    idx = np.argmax(probs)

    if idx == 0:
        intent = "Greeting"
        keyword = None
    elif idx == 1:
        intent = "Search"
        keyword = getEntities(keyboard)
    elif idx == 2:
        intent = "Suggestions"
        keyword = getEntities(keyboard)
    elif idx == 3:
        intent = "Farewell"
        keyword = None

    return intent, keyword

In [ ]:
Intent_detection_function()